In [28]:
import streamlit as st
import sqlite3
import datetime
import re
import os
from datetime import datetime, timedelta
import dateparser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from langchain.chains import LLMChain


In [29]:
def init_db():
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    
    # Check if the table exists
    c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='appointments'")
    table_exists = c.fetchone()
    
    if table_exists:
        # Check if email column exists
        try:
            c.execute("SELECT email FROM appointments LIMIT 1")
        except sqlite3.OperationalError:
            # Add email column if it doesn't exist
            print("Adding email column to existing database...")
            c.execute("ALTER TABLE appointments ADD COLUMN email TEXT DEFAULT 'no-email@example.com'")
            conn.commit()
    else:
        # Create new table with email column
        c.execute('''
        CREATE TABLE appointments
        (id INTEGER PRIMARY KEY AUTOINCREMENT,
         name TEXT NOT NULL,
         email TEXT NOT NULL,
         date TEXT NOT NULL,
         time TEXT NOT NULL,
         purpose TEXT,
         created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)
        ''')
        conn.commit()
    
    conn.close()

init_db()


In [30]:
def add_appointment(name, email, date, time, purpose):
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    c.execute("INSERT INTO appointments (name, email, date, time, purpose) VALUES (?, ?, ?, ?, ?)",
              (name, email, date, time, purpose))
    conn.commit()
    conn.close()

def get_appointments(name=None, email=None, date=None):
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    
    query = "SELECT * FROM appointments"
    params = []
    
    if name and email and date:
        query += " WHERE name = ? AND email = ? AND date = ?"
        params = [name, email, date]
    elif name and email:
        query += " WHERE name = ? AND email = ?"
        params = [name, email]
    elif name and date:
        query += " WHERE name = ? AND date = ?"
        params = [name, date]
    elif email and date:
        query += " WHERE email = ? AND date = ?"
        params = [email, date]
    elif name:
        query += " WHERE name = ?"
        params = [name]
    elif email:
        query += " WHERE email = ?"
        params = [email]
    elif date:
        query += " WHERE date = ?"
        params = [date]
    
    query += " ORDER BY date, time"
    
    c.execute(query, params)
    appointments = c.fetchall()
    conn.close()
    return appointments


In [31]:
add_appointment("John Doe", "john@example.com", "2025-03-20", "14:30", "Project discussion")
print("Appointment added successfully!")


Appointment added successfully!


In [32]:
appointments = get_appointments(name="John Doe")
print(appointments)


[(4, 'John Doe', '2025-03-20', '14:30', 'Project discussion', '2025-03-14 06:11:18', 'john@example.com'), (5, 'John Doe', '2025-03-20', '14:30', 'Project discussion', '2025-03-14 08:40:25', 'john@example.com'), (6, 'John Doe', '2025-03-20', '14:30', 'Project discussion', '2025-03-14 08:52:40', 'john@example.com')]


In [33]:
def check_appointment_exists(name, email, date, time):
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    
    try:
        c.execute("SELECT * FROM appointments WHERE name = ? AND email = ? AND date = ? AND time = ?", 
                (name, email, date, time))
        result = c.fetchone()
    except sqlite3.OperationalError:
        c.execute("SELECT * FROM appointments WHERE name = ? AND date = ? AND time = ?", 
                (name, date, time))
        result = c.fetchone()
    
    conn.close()
    return result is not None


In [34]:
exists = check_appointment_exists("John Doe", "john@example.com", "2025-03-20", "14:30")
print("Appointment exists?" , exists)


Appointment exists? True


In [35]:
def check_appointment_exists(name, email, date, time):
    conn = sqlite3.connect('appointments.db')
    c = conn.cursor()
    
    try:
        c.execute("SELECT * FROM appointments WHERE name = ? AND email = ? AND date = ? AND time = ?", 
                (name, email, date, time))
        result = c.fetchone()
    except sqlite3.OperationalError:
        c.execute("SELECT * FROM appointments WHERE name = ? AND date = ? AND time = ?", 
                (name, date, time))
        result = c.fetchone()
    
    conn.close()
    return result is not None


In [44]:
def setup_llm():
    api_key = os.environ.get('GEMINI_API_KEY')  # Replace with actual key
    
    if not api_key:
        raise ValueError("Google API key not found. Set it in your environment variables.")
    
    llm = ChatGoogleGenerativeAI(
        api_key=api_key,
        model="gemini-2.0-flash"
    )
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an appointment booking assistant. Your task is to:
        1. Help users book appointments by extracting name, email, date, time, and purpose.
        2. Help users retrieve their appointment information.
        3. Respond in a friendly and helpful manner.
        
        Format your response with JSON-like tags:
        <APPOINTMENT_DETAILS>
        name: [extracted name]
        email: [extracted email]
        date: [extracted date in YYYY-MM-DD format]
        time: [extracted time in HH:MM format]
        purpose: [extracted purpose]
        action: [book/retrieve]
        </APPOINTMENT_DETAILS>
        """),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ])
    
    memory = ConversationBufferMemory(return_messages=True, input_key="input", memory_key="history")
    
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        memory=memory
    )
    
    return chain


In [46]:
llm_chain = setup_llm()
llm_chain.invoke("tell me about teh services")

{'input': 'tell me about teh services',
 'history': [HumanMessage(content='tell me about teh services', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I can help you book or retrieve information about appointments. To proceed, could you please specify what you would like to do?', additional_kwargs={}, response_metadata={})],
 'text': 'I can help you book or retrieve information about appointments. To proceed, could you please specify what you would like to do?'}

In [47]:
llm_chain.invoke("retrieve information about the appointments")

{'input': 'retrieve information about the appointments',
 'history': [HumanMessage(content='tell me about teh services', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I can help you book or retrieve information about appointments. To proceed, could you please specify what you would like to do?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='retrieve information about the appointments', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Okay, I can help with that! To retrieve your appointment information, I'll need your name and email address. Could you please provide those?", additional_kwargs={}, response_metadata={})],
 'text': "Okay, I can help with that! To retrieve your appointment information, I'll need your name and email address. Could you please provide those?"}

In [48]:
llm_chain.invoke("my name is zabihullah and email address is zabihullah18381@gmail.com")

{'input': 'my name is zabihullah and email address is zabihullah18381@gmail.com',
 'history': [HumanMessage(content='tell me about teh services', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I can help you book or retrieve information about appointments. To proceed, could you please specify what you would like to do?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='retrieve information about the appointments', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Okay, I can help with that! To retrieve your appointment information, I'll need your name and email address. Could you please provide those?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is zabihullah and email address is zabihullah18381@gmail.com', additional_kwargs={}, response_metadata={}),
  AIMessage(content="```json\n<APPOINTMENT_DETAILS>\nname: zabihullah\nemail: zabihullah18381@gmail.com\ndate: null\ntime: null\npurpose: null\nactio

In [38]:
def extract_appointment_details(response_text):
    details_pattern = r'<APPOINTMENT_DETAILS>(.*?)</APPOINTMENT_DETAILS>'
    match = re.search(details_pattern, response_text, re.DOTALL)
    
    if not match:
        return None, response_text.strip()
    
    details_text = match.group(1).strip()
    details = {}
    
    name_match = re.search(r'name:\s*(.*)', details_text)
    email_match = re.search(r'email:\s*(.*)', details_text)
    date_match = re.search(r'date:\s*(.*)', details_text)
    time_match = re.search(r'time:\s*(.*)', details_text)
    purpose_match = re.search(r'purpose:\s*(.*)', details_text)
    action_match = re.search(r'action:\s*(.*)', details_text)
    
    if name_match:
        details['name'] = name_match.group(1).strip()
    if email_match:
        details['email'] = email_match.group(1).strip()
    if date_match:
        date_str = date_match.group(1).strip()
        parsed_date = dateparser.parse(date_str)
        details['date'] = parsed_date.strftime('%Y-%m-%d') if parsed_date else date_str
    if time_match:
        details['time'] = time_match.group(1).strip()
    if purpose_match:
        details['purpose'] = purpose_match.group(1).strip()
    if action_match:
        details['action'] = action_match.group(1).strip()
    
    clean_response = re.sub(details_pattern, '', response_text, flags=re.DOTALL).strip()
    
    return details, clean_response


In [39]:
def process_message(user_input, llm_chain):
    try:
        llm_response = llm_chain.invoke({"input": user_input})
        response_text = llm_response["text"]
        details, clean_response = extract_appointment_details(response_text)

        if details:
            return details
        return clean_response
    
    except Exception as e:
        return f"Error: {str(e)}"


In [40]:
user_input = "Book an appointment for John Doe with email john@example.com on March 20, 2025, at 2:30 PM for a project discussion."
response = process_message(user_input, llm_chain)
print(response)


Error: Missing keys ['session_id'] in config['configurable'] Expected keys are ['session_id'].When using via .invoke() or .stream(), pass in a config; e.g., chain.invoke({'input': 'foo'}, {'configurable': {'session_id': '[your-value-here]'}})


In [41]:
user_input = "Do I have any appointments on March 20, 2025?"
response = process_message(user_input, llm_chain)
print(response)


Error: Missing keys ['session_id'] in config['configurable'] Expected keys are ['session_id'].When using via .invoke() or .stream(), pass in a config; e.g., chain.invoke({'input': 'foo'}, {'configurable': {'session_id': '[your-value-here]'}})
